In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
ret, frame = cap.read()

# cascade 
cascade = cv2.CascadeClassifier(r'C:\Users\Data\computer vision\haarcascades\haarcascade_frontalface_default.xml')
face_rect = cascade.detectMultiScale(frame)


(x,y,w,h) = tuple(face_rect[0])
track_window= (x,y,w,h)


# Roi (region of interes)
roi = frame[y:y + h, x:x + w]

# hsv color mapping
roi_hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# roi hist
roi_hist = cv2.calcHist([roi_hsv], [0], None, [180], [0, 180])

# normaize 
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)


# determinan criteria
term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10 ,1)

while True:
    ret, frame = cap.read()
    
    if ret == True:
        
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # back Project
        dst = cv2.calcBackProject([hsv],[0], roi_hist, [0,180], 2)
        
        # apply camshift
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
        # show
        pts = cv2.boxPoints(ret)
        pts = np.intp(pts)
        img2 = cv2.polylines(frame,[pts], True, (0,255,0), 5)
        img2 = cv2.imshow('img2', img2)
        
        k = cv2.waitKey(10) & 0xff
        if k==27:
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()
        